In [ ]:
import random
import numpy as np
import tensorflow as tf

SEED=1995

np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

gru_tf = tf.keras.layers.GRU(
    units=5,
    return_sequences=True,
    kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED),
    recurrent_initializer=tf.keras.initializers.Orthogonal(seed=SEED),
    bias_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)
)

y_tf = gru_tf(tf.ones((1, 3, 5)), training=False)  # forward pass with ones

np.savez(
    'tf_model_weights.npz', 
    gru_kernel=gru_tf.weights[0].numpy(), 
    gru_recurrent_kernel=gru_tf.weights[1].numpy(),
    gru_bias=gru_tf.weights[2].numpy()
)

In [ ]:
import random as r
import numpy as np
import torch

SEED=1995
torch.set_printoptions(precision=8)

r.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# from speechbrain.nnet.RNN import GRU, LSTM

npz_weights = np.load('tf_model_weights.npz')


def convert_input_kernel_inv(kernel):
    kernel_r, kernel_z, kernel_h = np.hsplit(kernel, 3)
    return np.concatenate((kernel_z.T, kernel_r.T, kernel_h.T))
    

def convert_recurrent_kernel_inv(kernel):
    kernel_r, kernel_z, kernel_h = np.hsplit(kernel, 3)
    return np.concatenate((kernel_z.T, kernel_r.T, kernel_h.T))


def convert_bias_inv(bias):
    bias = bias.reshape(2, 3, -1) 
    return bias[:, [1, 0, 2], :].reshape((2, -1))


gru_pt = torch.nn.GRU(
    hidden_size=5,
    input_size=5,
    num_layers=1,
    bidirectional=False,
    batch_first=True
)
for pn, p in gru_pt.named_parameters():
    if 'weight_ih' in pn:
        p.data = torch.from_numpy(convert_input_kernel_inv(npz_weights['gru_kernel']))
    elif 'weight_hh' in pn:
        p.data = torch.from_numpy(convert_recurrent_kernel_inv(npz_weights['gru_recurrent_kernel']))
    elif 'bias_ih' in pn:
        p.data = torch.from_numpy(convert_bias_inv(npz_weights['gru_bias'])[0])
    else:
        p.data = torch.from_numpy(convert_bias_inv(npz_weights['gru_bias'])[1])

In [ ]:
y_tf

In [ ]:
gru_pt.eval()

In [ ]:
y_pt, _ = gru_pt(torch.ones(1, 3, 5))

In [ ]:
y_pt

In [ ]:
def convert_kernel(kernel):
    kernel_z, kernel_r, kernel_h = np.vsplit(kernel, 3)
    return np.concatenate((kernel_r.T, kernel_z.T, kernel_h.T), axis=1)

def convert_bias(bias):
    bias = bias.reshape(2, 3, -1) 
    return bias[:, [1, 0, 2], :].reshape((2, -1))

In [ ]:
for pn, p in gru_pt.named_parameters():
    if 'weight_ih' in pn:
        kernel = p.data
    elif 'weight_hh' in pn:
        recurrent_kernel = p.data
    elif 'bias_ih' in pn:
        bias_ih = p.data
    else:
        bias_hh = p.data
bias = np.stack((bias_ih, bias_hh), axis=0)

In [ ]:
bias = np.stack((bias_ih, bias_hh), axis=0)

In [ ]:
gru_tf.set_weights([convert_kernel(kernel), 
                    convert_kernel(recurrent_kernel), 
                    convert_bias(bias)])

In [ ]:
y_tf = gru_tf(tf.ones((1, 3, 5)), training=False)  # forward pass with ones

In [ ]:
y_tf

In [ ]:
# library imports:

import pandas as pd
from pathlib import Path
import portiloop_software
import torch
from portiloop_software import run_offline_unlabelled, get_final_model_config_dict, get_trained_model
from matplotlib import pyplot as plt
from torchsummary import summary
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Conv1D, MaxPool1D, GRU
from tensorflow.keras import Model
import numpy as np

In [ ]:
# path to the portiloop software package:

path_software = Path(portiloop_software.__file__).parent.absolute()

In [ ]:
# path to the folder containing pre-trained models:

path_experiments = path_software / 'experiments'

In [ ]:
# configuration dictionary of the model:

config_dict = get_final_model_config_dict()

In [ ]:
# run offline inference (on all data points):

model_torch = get_trained_model(config_dict, path_experiments)

In [ ]:
model_torch

In [ ]:
summary(model_torch)

In [ ]:
model_keras = tf.keras.Sequential()
model_keras.add(tf.keras.layers.Reshape((-1, 54, 1)))
model_keras.add(tf.keras.layers.Conv1D(31, strides=[1], kernel_size=7, activation='relu'))
model_keras.add(tf.keras.layers.MaxPooling2D(pool_size=(1, 7), strides=1, padding='valid'))
model_keras.add(tf.keras.layers.Conv1D(31, strides=[1], kernel_size=7, activation='relu'))
model_keras.add(tf.keras.layers.MaxPooling2D(pool_size=(1, 7), strides=1, padding='valid'))
model_keras.add(tf.keras.layers.Conv1D(31, strides=[1], kernel_size=7, activation='relu'))
model_keras.add(tf.keras.layers.MaxPooling2D(pool_size=(1, 7), strides=1, padding='valid'))
model_keras.add(tf.keras.layers.Reshape((-1, 558)))
model_keras.add(GRU(units=7, time_major=False))
model_keras.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model_keras.build((None, 50, 54))
model_keras.summary()

In [ ]:
model_keras.layers

In [ ]:
torch_params = [param.detach().numpy() for param in model_torch.parameters()]

In [ ]:
for i, param in enumerate(torch_params):
    print(f"param {i}: {param.shape}")

In [ ]:
def convert_kernel_inv(kernel):
    kernel_r, kernel_z, kernel_h = np.hsplit(kernel, 3)
#     print(f"kernel_r:{kernel_r}")
#     print(f"kernel_z:{kernel_z}")
#     print(f"kernel_h:{kernel_h}")
    return np.concatenate((kernel_z.T, kernel_r.T, kernel_h.T))

def convert_kernel(kernel):
    kernel_z, kernel_r, kernel_h = np.vsplit(kernel, 3)
#     print(f"kernel_r:{kernel_r}")
#     print(f"kernel_z:{kernel_z}")
#     print(f"kernel_h:{kernel_h}")
    return np.concatenate((kernel_r.T, kernel_z.T, kernel_h.T), axis=1)

def convert_bias(bias):
    bias = bias.reshape(2, 3, -1) 
    return bias[:, [1, 0, 2], :].reshape((2, -1))

In [ ]:
model_keras.layers[8].weights

In [ ]:
np.stack((torch_params[8], torch_params[9]), axis=0)

In [ ]:
torch_params[6].shape

In [ ]:
torch_params[7].shape

In [ ]:
l = [m for m in model_torch.modules()]
gru_pt = l[14]
gru_pt

In [ ]:
for pn, p in gru_pt.named_parameters():
    if 'weight_ih' in pn:
        kernel = p.data
    elif 'weight_hh' in pn:
        recurrent_kernel = p.data
    elif 'bias_ih' in pn:
        bias_ih = p.data
    else:
        bias_hh = p.data
bias = np.stack((bias_ih, bias_hh), axis=0)

In [ ]:
model_keras.layers[1].set_weights([torch_params[0].T, torch_params[1].T])

# Second Conv Layer:
model_keras.layers[3].set_weights([torch_params[2].T, torch_params[3].T])

# Third Conv Layer:
model_keras.layers[5].set_weights([torch_params[4].T, torch_params[5].T])

# GRU Layer:

# kernel_input = convert_kernel(torch_params[6])
# kernel_h = convert_kernel(torch_params[7])
# bias = convert_bias(np.stack((torch_params[8], torch_params[9]), axis=0))

# model_keras.layers[8].set_weights([kernel_input, 
#                                    kernel_h, 
#                                    bias])

model_keras.layers[8].set_weights([convert_kernel(kernel), 
                    convert_kernel(recurrent_kernel), 
                    convert_bias(bias)])

# Dense Layer:
model_keras.layers[9].set_weights([torch_params[10].T, torch_params[11].T])

In [ ]:
model_keras.layers[8].weights

In [ ]:
input_numpy = np.ones((1, 50, 54))
input_torch = torch.ones((1, 50, 54))

In [ ]:
x, hn1, _, _ = model_torch(input_torch, None, None, torch.zeros(1, 1, 7), None)

In [ ]:
x

In [ ]:
hn1

In [ ]:
out_keras = model_keras(input_numpy)
print(out_keras)

In [ ]:
from keras import backend as K

inp = model_keras.input                                           # input placeholder
outputs = [layer.output for layer in model_keras.layers]          # all layer outputs
functors = [K.function([inp], [out]) for out in outputs]   # evaluation function

# Testing
layer_outs = [func([input_numpy]) for func in functors]

In [ ]:
print("===")
print(np.array(layer_outs[2]).squeeze().swapaxes(1,2).shape)
print(np.array(layer_outs[2]).squeeze().swapaxes(1,2))

print("===")
print(np.array(layer_outs[4]).squeeze().swapaxes(1,2).shape)
print(np.array(layer_outs[4]).squeeze().swapaxes(1,2))

print("===")
print(np.array(layer_outs[6]).squeeze().swapaxes(1,2).shape)
print(np.array(layer_outs[6]).squeeze().swapaxes(1,2))

print("===")
print(np.array(layer_outs[8]).squeeze().shape)
print(np.array(layer_outs[8]).squeeze())

In [ ]:
gru_tf = model_keras.layers[8]
gru_tf

In [ ]:
gru_pt

In [ ]:
gru_pt(torch.ones(1, 50, 558))

In [ ]:
y_tf = gru_tf(tf.ones((1, 50, 558)), training=False)

In [ ]:
y_tf